## Swedcham Debit Note Automation

1. Read American Credit Card Invoice
2. Gather all needed pieces of information
2. Create a new Word File with the current Debit Note
3. Generate a PDF from the Word File
4. Send a mail to Swedcham

#### 1. Read American Credit Card Invoice

In [1]:
# Import Libraries
import PyPDF2
import datetime
import os
import re

In [47]:
# Define Script Variables

# Get American Credit Card Invoice folder
cc_folder = 'D:\\Primeway\\Users Pw\\Oscar.Carvalho\\Contas\\Cartões de Crédito\\American\\'
cc_pass = 'LaylaLaylaLayla'

# Get Destination Folder and Template
nd_folder = 'D:\\Primeway\\Primeway\\Administração\\Contas a Receber\\Nota de Débito\\'
nd_template = 'Primeway - Nota de Débito - Template.docx'
nd_recipient = 'elaine@swedcham.com.br'
nd_sender = 'oscar@primeway.com.br'

# Charge parameters
nd_description = "Plano Mensal MAILCHIMP - US$ {} (cotação R$ {} + 6,38% IOF + Comissão 10%)"
nd_IOF = 0.0638
nd_comission = 0.1


In [3]:
# Get last invoice file name

# Get current year
today = datetime.date.today()
year = str(today.year)
# print(year)

path = cc_folder + year + '\\'

file_list = [entry.name for entry in os.scandir(path) if entry.name[0:4] == year]
filename = file_list[len(file_list)-1]

print(path)
print(filename)

D:\Primeway\Users Pw\Oscar.Carvalho\Contas\Cartões de Crédito\American\2023\
2023-07-Extrato-American-Bradesco.pdf


In [4]:
# Open PDF file and get Mailchimp line
filename = '2023-07-Extrato-American-Bradesco.pdf'

with open(path + filename, 'rb') as obj:
   pdfReader = PyPDF2.PdfReader(obj)
   if pdfReader.is_encrypted:
      pdfReader.decrypt(cc_pass)
   page_txt = pdfReader.pages[0].extract_text()

pattern = '[0-9]+/[0-9]+/[0-9]+.+' + 'MAILCHIMP.+' + '\\n'
mailchimp_line = re.search(pattern, page_txt)
if (not mailchimp_line == None):
   mailchimp_line = mailchimp_line.group().replace(',', '.')
   print(mailchimp_line)
else:
   print('Mailchimp line not found!')

19/06/2023 MAILCHIMP 55.00 USD 55.00 5.0300 276.65



#### 2. Gather all needed pieces of information

In [48]:
# Get Mailchimp variables

# charging_date = re.search('[0-9]+/[0-9]+/[0-9]+', mailchimp_line).group()
# print(charging_date)

items = re.split('[ |\\n]', mailchimp_line)
nd_date_of_charge = items[0]
nd_value_usd      = items[4]
nd_value_brlusd   = items[5]
nd_value_brl      = items[6]

# Calc totals
nd_total = round(float(nd_value_usd) * float(nd_value_brlusd) * (1 + nd_IOF) * (1 + nd_comission), 2)
nd_total = str(nd_total).replace('.', ',')

# Format values
nd_value_usd    = str(nd_value_usd).replace('.', ',')
nd_value_brlusd = str(round(float(nd_value_brlusd), 2)).replace('.', ',')

nd_description = nd_description.format(nd_value_usd, nd_value_brlusd)
print(nd_description)

Plano Mensal MAILCHIMP - US$ 55,00 (cotação R$ 5,03 + 6,38% IOF + Comissão 10%)
